In [1]:
from ase import Atoms
import ase.units as units
from ase.io import read
from ase.build import bulk
from ase.spacegroup import crystal
from ase.calculators.espresso import Espresso

import numpy as np
import matplotlib.pyplot as plt

import os
import dotenv

dotenv.load_dotenv()
PWSCF_COMMAND = os.environ.get("PWSCF_COMMAND")
PPDIR = os.environ.get("PSEUDOPOTENTIALS")

outdir = "./results"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
pseudopotentials = {
    'Pb': 'Pb.pz-d-van.UPF',
    'Ti': 'Ti.pz-sp-van_ak.UPF',
    'O': 'O.pz-rrkjus.UPF'
}  

# Quantum Espresso parameters
qe_params = {
    'ecutwfc': 30,
    'ecutrho': 300,
    'kpts': (3, 3, 3),
    
    'calculation': 'scf',
    'pseudo_dir': PPDIR,
}

# Initialize arrays to store results
energies = []

# Constants
a_exp = 3.97  # Experimental lattice constant in Angstrom
a_range = 0.5  # Range to vary lattice constant
n_points = 20  # Number of points in the lattice constant range

# Define lattice parameters to sample
lattice_params = np.linspace(a_exp - a_range, a_exp + a_range, n_points)

# Loop over lattice parameters
for a in lattice_params:
    # Define the crystal structure
    atoms = crystal(['Pb', 'Ti', 'O', 'O', 'O'],
                    basis=[(0, 0, 0), (0.5, 0.5, 0.5), (0, 0.5, 0.5), (0.5, 0, 0.5), (0.5, 0.5, 0)],
                    spacegroup=221,
                    cellpar=[a, a, a, 90, 90, 90])

    # Set up Quantum Espresso calculator
    calc = Espresso(parameters=qe_params)
    atoms.set_calculator(calc)

    # Calculate energy
    energy = atoms.get_potential_energy()
    energies.append(energy)


In [ ]:

# Plotting
plt.plot(lattice_params, energies, '-o')
plt.xlabel('Lattice Parameter (Å)')
plt.ylabel('Energy (eV)')
plt.title('Energy vs Lattice Parameter for Cubic PbTiO3')
plt.show()